In [1]:
# importing libraries
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.framework import ops
import pandas as pd
from keras.utils.np_utils import to_categorical

/home/vajir/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/vajir/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/vajir/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/vajir/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
# importing data set
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [3]:
# train data info
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 251.5 MB


In [4]:
X_train = train_data.iloc[: ,1:].values
Y_train = train_data.iloc[: ,0].values
X_test = test_data.iloc[: ,:].values
print('X_train shape =' ,X_train.shape)
print('Y_train shape =' ,Y_train.shape)
print('X_test shape =' ,X_test.shape)

X_train shape = (42000, 784)
Y_train shape = (42000,)
X_test shape = (28000, 784)


In [5]:
from sklearn.cross_validation import train_test_split
X_train_std ,X_cv_std ,Y_train_std ,Y_cv_std = train_test_split(X_train ,Y_train ,test_size =0.2 ,random_state =0)
print(X_train_std.shape)
print(X_cv_std.shape)
print(Y_train_std.shape)
print(Y_cv_std.shape)

/home/vajir/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/vajir/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/vajir/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


(33600, 784)
(8400, 784)
(33600,)
(8400,)


In [54]:
# data preprocessing  
X_train_std = (np.transpose(X_train_std))/255.0
X_cv_std = (np.transpose(X_cv_std))/255.0
X_test_std = (np.transpose(X_test))/255.0
Y_train_std = np.transpose(to_categorical(Y_train_std ,num_classes =10))
Y_cv_std = np.transpose(to_categorical(Y_cv_std ,num_classes =10))
print(X_train_std.shape)
print(Y_train_std.shape)

(784, 33600)
(10, 33600)


In [55]:
# creating placeholder 
def create_placeholder(n_x ,n_y):
    X = tf.placeholder(tf.float32 ,shape=[n_x ,None] )
    Y = tf.placeholder(tf.float32 ,shape =[n_y ,None] )
    return X ,Y

In [56]:
# network architecture
tf.reset_default_graph()
def initialize_parameter():
    layer = np.array((784 ,500,100 ,10))
    parameters ={}
    for i in range(0 ,len(layer)-1):
        parameters['W' + str(i+1)] = tf.get_variable(('W' +str(i+1)) ,[layer[i+1], layer[i]] ,initializer= tf.contrib.layers.xavier_initializer())
        parameters['b' + str(i+1)] = tf.get_variable('b' +str(i+1) ,[layer[i+1] ,1] ,initializer = tf.zeros_initializer())
        
    return parameters

In [57]:
# forward loop
def forward_prop(X ,parameters ,layer):
    a_prev =X
    linear_cache ={}
    activation_cache ={}
    for i in range(0 ,len(layer)-1):
        linear_cache['Z' +str(i+1)] = tf.add(tf.matmul(parameters['W' +str(i+1)] ,a_prev) ,parameters['b' +str(i+1)])
        activation_cache['A' + str(i+1)] = tf.nn.relu(linear_cache['Z' +str(i+1)])
        a_prev = activation_cache['A' +str(i+1)]
    return linear_cache

In [58]:
# computing cost  Y -(num_classes ,m)
def compute_cost(X ,parameters ,layer ,Y):
    linear_cache = forward_prop(X ,parameters ,layer)
    logits = tf.transpose(linear_cache['Z' +str(len(layer) -1)])
    labels = tf.transpose(Y)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits =logits ,labels = labels))
    return cost

In [59]:
tf.reset_default_graph()
def compute_cost_with_regularization(X,parameters,layer ,Y ,lambd ,m):
    entropy_cost = compute_cost(X ,parameters ,layer ,Y)
    entropy_cost = tf.cast(entropy_cost, tf.float32)
    L2_cost = 0.0
    for i in range(1 ,len(layer)):
        L2_cost = L2_cost + tf.reduce_sum(tf.square(parameters['W' + str(i)]))
    L2_cost = tf.multiply((tf.cast(L2_cost, tf.float32)) , lambd/(2.0*m))
    cost = tf.cast(entropy_cost + L2_cost ,tf.float32)
    return cost


In [60]:
tf.reset_default_graph()
def model(X_train_std ,Y_train_std,learning_rate =0.01,num_epoch =11480 ,lambd =0.6):
    n_x = 784 
    n_y = 10
    m = X_train_std.shape[1]
    costs =[]
    layer = np.array((784 ,500,100 ,10))
    X,Y = create_placeholder(n_x ,n_y)
    parameters =initialize_parameter()
    linear_cache = forward_prop(X ,parameters ,layer)
    cost = compute_cost_with_regularization(X ,parameters ,layer ,Y ,lambd ,m)
    optimizer = tf.train.RMSPropOptimizer(learning_rate =learning_rate).minimize(cost)
    init = tf.global_variables_initializer()      # intitalizing the global variables
    sess = tf.Session()
    sess.run(init)
    for epoch in range(num_epoch):
        
        _ ,epoch_cost = sess.run([optimizer ,cost] ,feed_dict ={X:X_train_std ,Y:Y_train_std})
        if epoch% 5 == 0 :
            print('cost after %d epoch is %f '%(epoch ,epoch_cost))
        costs.append(epoch_cost)
    plt.plot(np.squeeze(costs))         #  plot graph between cost and num of iteration
    plt.ylabel('cost')
    plt.xlabel('iterations (per tens)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    parameters = sess.run(parameters)      # saving parameters
    sess.close()
    return parameters

In [ ]:
tf.reset_default_graph()
parameters = model(X_train_std ,Y_train_std)

In [62]:
def accuracy(X_train_std ,Y_train_std , parameters):
    n_x =784
    n_y = 10
    layer = np.array((784 ,500,100 ,10))
    X ,Y = create_placeholder(n_x ,n_y)
    cache = forward_prop(X ,parameters ,layer)
    sess= tf.Session()
    linear_cache = sess.run(cache ,feed_dict ={X:X_train_std})
    Y_pred = linear_cache['Z' +str(len(layer)-1)]
    correct_prediction = tf.equal(tf.argmax(Y_pred), tf.argmax(Y_train_std))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction ,'float'))
    acc = sess.run(accuracy)
    return acc
    

In [63]:
train_acc = accuracy(X_train_std ,Y_train_std ,parameters)
print('train accuracy:' + str(train_acc))

train accuracy:0.98854166


In [64]:
cross_val_acc = accuracy(X_cv_std ,Y_cv_std ,parameters)
print('cross validation accuracy:' + str(cross_val_acc))

cross validation accuracy:0.9697619


In [65]:
def predict(X_test_std ,parameters):
    n_x =784
    n_y = 10
    layer = np.array((784 ,500,100 ,10))
    X ,Y = create_placeholder(n_x ,n_y)
    sess= tf.Session()
    cache = forward_prop(X ,parameters ,layer)
    linear_cache = sess.run(cache ,feed_dict ={X:X_test_std})
    Y_pred = linear_cache['Z' +str(len(layer)-1)]
    Y_test = np.argmax(Y_pred ,axis =0)
    sess.close()
    return Y_test


In [66]:
Y_test_pred = predict(X_test_std ,parameters)

In [67]:
print(Y_test_pred.shape)

(28000,)


In [85]:
data1 = pd.read_csv('sample_submission.csv')
Y1 = data1.iloc[: ,0].values
Y = {'ImageId':Y1 ,'Label':Y_test_pred}
df = pd.DataFrame(data =Y ,dtype =int)
df.to_csv('Data_recog_submission.csv' ,sep =',')